## Загрузка необходимых библиотек

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file

# Библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Библиотеки для обработки дат
import re
from datetime import datetime, timedelta
import time

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# Загружаем данные
data = pd.read_csv('main_task.csv')

In [ ]:
data.head()

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

## 1. Обработка NAN 

По информации о полученных данных мы видим, что в 3-х столбцах есть пропущенные значение: 'Cuisine Style', 'Price Range', 'Number of Reviews'. Эти пропуски необходимо обработкать, заполнить какими-то значениями или удалить.

In [ ]:
# Функция заполняет пропуски в столбце 'Number of Reviews' средним значением по городу
def fillna_number_reviews(row):
    if np.isnan(row['Number of Reviews']):
        row['Number of Reviews'] = round(data[data['City'] == row['City']]['Number of Reviews'].mean(),0)
        return row['Number of Reviews']
    return row['Number of Reviews']

In [ ]:
# Применяем функция для заполнения пропусков
data['Number of Reviews'] = data.apply(lambda row: fillna_number_reviews(row), axis=1)

In [ ]:
# Заменяем пропуски в 'Cuisine Style' на значение 'Not data'
data['Cuisine Style'].fillna("['Not data']", axis=0, inplace=True)

In [ ]:
# Заменяем пропуски в 'Price Range' на значение 0
data['Price Range'].fillna('0', axis=0, inplace=True)

## 2. Обработка признаков и создание новых

***Рассмотрим признак "Price Range"***

In [ ]:
data['Price Range'].value_counts()

In [ ]:
# Преобразовыем категориальные данные в числовой формат
data['Price Range'] = data['Price Range'].map({'$$ - $$$': 2, '$': 1, '$$$$':3, '0':0})

In [ ]:
# Посмотрим на распределение признака
plt.rcParams['figure.figsize'] = (10,7)
data['Price Range'].hist()

***Обработка признака 'Cuisine Style'***

In [ ]:
# Пребразование строки в список стилей кухонь, представленных в ресторанах
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: x[2:-2])
data['Cuisine Style'] = data['Cuisine Style'].str.split("', '")

In [ ]:
# Создаем множество стилей кухонь
cuisins = set()
for cuisins_style in data['Cuisine Style']:
    for cuisin in cuisins_style:
        cuisins.add(cuisin)
        
len(cuisins) #кол-во стилей кухонь, +1 - нет данных        

In [ ]:
# Создаем функцию для преобразования в dummy-переменные
def find_item(cell):
    if item in cell:
        return 1
    return 0

In [ ]:
# Применяем функцию для преобразования 'Cuisine Style' в dummy-переменные
for item in cuisins:
    data[item] = data['Cuisine Style'].apply(find_item)

In [ ]:
# Удаляем столбцы с наименьшей информативностью
columns_cuisins_drop = [s for s in cuisins if data[s].sum() < 50]
data.drop(columns_cuisins_drop, axis = 1, inplace=True)

In [ ]:
# Преобразеум признак 'Cuisine Style' в признак с количеством стилей кухонь, представленных в ресторанах
data['Cuisine Style'] = data['Cuisine Style'].apply(lambda x: len(x))

In [ ]:
data.info()

***Обработка признака Reviews***

In [ ]:
# Выделяем значения дат отзывов в остельный столбец
data['Date reviews'] = data['Reviews'].apply(lambda x: re.findall(r'\d{2}/\d{2}/\d{4}', x))

In [ ]:
# Преобразовываем строки с информацией о датах отзывов в список со значениями в формате datetime
for i in range(len(data['Date reviews'])):
    data['Date reviews'][i] = [datetime.strptime(x, '%m/%d/%Y') for x in data['Date reviews'][i]]

In [ ]:
# Для удобства вычисления преобразуем даты в формат "кол-во дней с 1970 года"
for i in range(len(data['Date reviews'])):
    data['Date reviews'][i] = list(map(datetime.timestamp, data['Date reviews'][i]))

In [ ]:
# Добавляем признак даты последнего отзыва, в формате "кол-во дней с 1970 года"
data['Date last review'] = data.apply(lambda x: max(x['Date reviews']) if len(x['Date reviews']) > 0 else 0, axis=1)

In [ ]:
# Посмотрим на распределение признака
data['Date last review'].hist(bins=100)

***Обработка признака City и добавление новых признаков***

In [ ]:
data['City'].value_counts()

In [ ]:
# Посмотрим на распределение признака
data['City'].hist(bins=100)

In [ ]:
# Создаем новый признак с id города, на основании телефонного кода
data['City ID'] = data['City'].map({'London': 4420, 'Paris': 331, 'Madrid':341, 'Barcelona':343,
                                   'Berlin':4930, 'Milan':392, 'Rome':396, 'Prague':4202,
                                   'Lisbon':35121, 'Vienna':431, 'Amsterdam':3120, 'Brussels':322,
                                   'Hamburg':4940, 'Munich':4989, 'Lyon':33437, 'Stockholm':468,
                                   'Budapest':361, 'Warsaw':4822, 'Dublin':3531, 'Copenhagen':451,
                                   'Athens':30210, 'Edinburgh':44131, 'Zurich':411, 'Oporto':3512,
                                   'Geneva':4122, 'Krakow':4812, 'Oslo':4722, 'Helsinki':3589,
                                   'Bratislava':4212, 'Luxembourg':352, 'Ljubljana':3861})

In [ ]:
# Создаем новый признак с населением города, млн.чел.
data['City population'] = data['City'].map({'London': 8.982, 'Paris': 2.140, 'Madrid':6.642, 'Barcelona':5.575,
                                           'Berlin':3.769, 'Milan':1.332, 'Rome':2.870, 'Prague':1.309,
                                           'Lisbon':0.507, 'Vienna':1.897, 'Amsterdam':0.825, 'Brussels':1.209,
                                           'Hamburg':1.841, 'Munich':1.472, 'Lyon':0.496, 'Stockholm':0.976,
                                           'Budapest':1.752, 'Warsaw':1.791, 'Dublin':1.388, 'Copenhagen':1.247,
                                           'Athens':3.169, 'Edinburgh':0.525, 'Zurich':0.402, 'Oporto':0.222,
                                           'Geneva':0.499, 'Krakow':0.779, 'Oslo':0.681, 'Helsinki':0.655,
                                           'Bratislava':0.438, 'Luxembourg':0.122, 'Ljubljana':0.284}) 

In [ ]:
# Создаем новый с площадью города, км.кв.
data['City square'] = data['City'].map({'London': 1572, 'Paris': 105.4, 'Madrid':604.3, 'Barcelona':101.9,
                                       'Berlin':891.8, 'Milan':181.8, 'Rome':1285, 'Prague':496,
                                       'Lisbon':100, 'Vienna':414.6, 'Amsterdam':219.3, 'Brussels':32.61,
                                       'Hamburg':755.2, 'Munich':310.4, 'Lyon':47.87, 'Stockholm':188,
                                       'Budapest':525.2, 'Warsaw':517.2, 'Dublin':115, 'Copenhagen':88.25,
                                       'Athens':2929, 'Edinburgh':264, 'Zurich':87.88, 'Oporto':41.42,
                                       'Geneva':15.93, 'Krakow':327, 'Oslo':454, 'Helsinki':213.8,
                                       'Bratislava':367.6, 'Luxembourg':51.46, 'Ljubljana':163.8}) 

### Посмотрим распределение целевой переменной

In [ ]:
data['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
data['Ranking'][data['Rating'] == 5].hist(bins=100)

In [ ]:
data['Ranking'][data['Rating'] < 4].hist(bins=100)

### Корреляция признаков
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data[['Ranking', 'Rating', 'Price Range', 'Number of Reviews', 'Cuisine Style', 
                 'Date last review', 'City ID', 'City population', 'City square']].corr(),)

In [ ]:
# модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
object_columns = [s for s in data.columns if data[s].dtypes == 'object']
data.drop(object_columns, axis = 1, inplace=True)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = data.drop(['Rating'], axis=1)
y = data['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = np.round(regr.predict(X_test), 1)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))